In [13]:
import torch
from GPT import * 
from dataclasses import dataclass

In [25]:
@dataclass
class config:
    vocab_size: int = len(set(text))
    block_size: int = 8
    n_embed:    int = 12
    n_head:     int = 12
    bias:       bool = False
    dropout:  float = 0.2
    n_layer:    int = 12


In [26]:
with open("Data/dialog.txt","r") as f:
    data = f.read()
text = data.replace("|",":")
len(set(data))
words = sorted([i for i in set(data)])

In [27]:
idxtos = {idx:w for idx,w in enumerate(words)}
stoidx = {w:idx for idx,w in enumerate(words)}
encode = lambda s: [stoidx[i] for i in s]
decode = lambda i: "".join(idxtos[j] for j in i)

In [28]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [29]:
def get_batch(split,batch_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (batch_size,))
    x = torch.stack([data[i:i+config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+config.block_size+1] for i in ix])
    return x, y

In [30]:
model = GPT(config)

In [31]:
eval_iters = 200
max_iters = 2000
eval_interval = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split,32)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

IndexError: index out of range in self